# `DESS`: End-to-End Workflow

## Imports & Config

In [ ]:
import pandas as pd

# Importing custom utilities
import stats as stats
import data_pipeline_manager as dpm
import dess.search as search
import dess.nlp as nlp

In [ ]:
INPUT_FILE = 'storage/input.dta'
UPLOAD_FILE_PATH = 'storage/completed_DepartmenttoSearch_November2024.dta'
COMPLETE_FILE_PATH = 'storage/complete.parquet'
REPROCESS_FILE_PATH = 'storage/reprocess.parquet'
UNCOMPLETE_FILE_PATH = 'storage/uncomplete.parquet'

## Data Prereprocessing

We recieve a file from Dropbox that we pass through our data pipeline to merge into our internal existing parquet files. Also, in this stage, we add relevant columns to `uncompleted.parquet`.

In [ ]:
df_master = pd.read_stata(INPUT_FILE)
df_c = pd.read_parquet(COMPLETE_FILE_PATH)
df_r = pd.read_parquet(REPROCESS_FILE_PATH)

In [ ]:
stats.get_expected_file_split_stats(df_master, df_c, df_r)

In [ ]:
df_u = dpm.get_new_rows()
df_u

In [ ]:
df_u = dpm.prepare_dess_data_structure(df_u)
df_u

In [ ]:
dpm.write_to_file(UNCOMPLETE_FILE_PATH, df_u, overwrite=True)

## Processing Data
In our existing workflow, we have three high-level functions that should be modularized since they may be used in any order. These are:
- **Scrape** — search for faculty information (from Google)
- **Extract** — Based on snapshots from Google search results, extract relevant faculty information.
- **Merge** — Update our files with new information.

### Scraping
Run the processing script on `uncompleted.parquet`. Code block included below is meant for demonstrations purposes. It's preferable to run this process in a separate terminal window to easily track processing.

In [ ]:
caffeinate -dui python3 search.py [start_index]

To check the progress of the scraping script, run the following block. It will provide an update on number of chunks processed, and percentage of entire file processed so far.

In [ ]:
df_u = pd.read_parquet(UNCOMPLETE_FILE_PATH)
stats.get_chunk_processing_stats(df_u, CHUNK_SIZE=200)

### Department Extraction
- Call `populate_faculty` on file

In [ ]:
nlp.extract_department_information(df_u) # modifes relevant columns in place
df_u

### Data merging
Often the scraping process is run in parallel so we have utilites to stitch together the complete file.

In [ ]:
df_u_full = dpm.get_merged_data_from_parallel_scrape(pd.read_parquet('storage/uncomplete-akhil.parquet'),
                                                 pd.read_parquet('storage/uncomplete-shishir.parquet'))
dpm.write_to_file(UNCOMPLETE_FILE_PATH, df_u_full, overwrite=True)

We also want to merge the scraped and faculty-filled information to other internal files.

In [ ]:
# add rows to `completed.parquet` and `reprocessed.parquet` and remove rows from `uncompleted.parquet`
# Will provide error messages for any conflicts. Returns the updated dataframes
df_c, df_r = dpm.update_internal_files(df_c, df_r, df_u_full)

# TODO: write back
dpm.write_to_file(COMPLETE_FILE_PATH, df_c, overwrite=True)
dpm.write_to_file(REPROCESS_FILE_PATH, df_r, overwrite=True)

## Post-Processing
To geta status update about the dataset—i.e. to get an overview of the completion rates and conversion rates—run the following block.

In [ ]:
df_result = pd.read_parquet(COMPLETE_FILE_PATH)

In [ ]:
stats.get_dataset_stats('storage/result.parquet')

To backup the results (i.e. upload to Dropbox), run the following blocks:

In [ ]:
dpm.create_stata_output_file("completeNov.dta")

In [ ]:
dpm.orchestrate_upload_workflow()